In [5]:
import spacy
from torchtext import data
import torch
import torch.nn as nn

## Import Model

In [6]:
# declare model class
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        embedded = self.dropout(self.embedding(x))
        
        output, (hidden, cell) = self.rnn(embedded)
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
            
        return self.fc(hidden.squeeze(0))

In [7]:
# load vocabulary

PATH1 = 'vocab_win.pt'
comment = torch.load(PATH1)


PATH2 = "model.pt"
INPUT_DIM = len(comment.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5

# create model object
model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT)

# load saved model and evaluate
model.load_state_dict(torch.load(PATH2, map_location='cpu'))
model.eval()

RNN(
  (embedding): Embedding(25002, 100)
  (rnn): LSTM(100, 256, num_layers=2, dropout=0.5, bidirectional=True)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (dropout): Dropout(p=0.5)
)

## Predict Sentiment

In [8]:
NLP = spacy.load('en')
def predict_sentiment(sentence):
    tokenized = [tok.text for tok in NLP.tokenizer(sentence)] # tokenize input sentence
    indexed = [comment.vocab.stoi[t] for t in tokenized]      # index the tokens
    tensor = torch.LongTensor(indexed)                        # convert tokens to PyTorch tensors
    tensor = tensor.unsqueeze(1)                              # add batch dimension
    prediction = torch.sigmoid(model(tensor))                 # apply on linearity
    return prediction.item()                                  # converts tensor to af float

In [19]:
sent1 = "Your work was an utter embarassment, it was garbage."
print(sent1, "   :", predict_sentiment(sent1))
sent2 = "Your work was pretty awful, please do better next time."
print(sent2, ":", predict_sentiment(sent2))

Your work was an utter embarassment, it was garbage.    : 0.970153272151947
Your work was pretty awful, please do better next time. : 0.0577467642724514
